In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
from torch.utils.data import DataLoader


In [24]:
!pip install scikit-learn

     ---------------------------------------- 0.0/59.0 kB ? eta -:--:--
     ---------------------------------------- 0.0/59.0 kB ? eta -:--:--
     ------------------- ------------------ 30.7/59.0 kB 660.6 kB/s eta 0:00:01
     -------------------------------------- 59.0/59.0 kB 624.8 kB/s eta 0:00:00
   ---------------------------------------- 0.0/9.3 MB ? eta -:--:--
   -- ------------------------------------- 0.6/9.3 MB 18.2 MB/s eta 0:00:01
   ------- -------------------------------- 1.8/9.3 MB 23.5 MB/s eta 0:00:01
   ---------------- ----------------------- 3.8/9.3 MB 30.3 MB/s eta 0:00:01
   ---------------------------- ----------- 6.7/9.3 MB 35.5 MB/s eta 0:00:01
   ---------------------------------------  9.3/9.3 MB 42.2 MB/s eta 0:00:01
   ---------------------------------------- 9.3/9.3 MB 39.4 MB/s eta 0:00:00
   ---------------------------------------- 0.0/301.2 kB ? eta -:--:--
   ---------------------------------------- 301.2/301.2 kB ? eta 0:00:00
   ------------------

In [42]:
!pip install torchsummary

In [2]:
def ResNet18SimCLR():
    resnet = models.resnet18(pretrained=False)
    resnet.fc = nn.Identity()  # Remove the fully connected layer
    return resnet


In [3]:
def augment(x):
    transform = transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomApply([transforms.ColorJitter(0.8, 0.8, 0.8, 0.2)], p=0.8),
    ])
    x = torch.stack([transform(img) for img in x])
    return x

In [4]:
def projection_head(x_tuple, hidden_dim=256,device=torch.device("cuda" if torch.cuda.is_available() else "cpu")):
    flattened_tensors = [tensor.flatten(start_dim=1) for tensor in x_tuple]
    # Concatenate the flattened tensors along dimension 1
    concatenated_tensor = torch.cat(flattened_tensors, dim=1)
    projection = nn.Sequential(
        nn.Linear(concatenated_tensor.shape[1], hidden_dim),
        nn.ReLU(inplace=True),
        nn.Linear(hidden_dim, hidden_dim),
    ).to(device)  # Move projection head to the correct device
    return projection(concatenated_tensor)

In [5]:
def NTXentLoss(z1, z2, temperature=0.5):
    N = z1.size(0)
    z = torch.cat([z1, z2], dim=0)
    sim = torch.mm(z, z.t()) / (torch.norm(z, dim=1, keepdim=True) * torch.norm(z.t(), dim=0, keepdim=True))
    sim = sim / temperature
    sim_exp = torch.exp(sim)
    sim_exp = sim_exp - torch.eye(2 * N, device=z.device)
    sim_1_2 = sim_exp[:N, N:]
    sim_2_1 = sim_exp[N:, :N]
    sim_1_1 = sim_exp[:N, :N]
    sim_2_2 = sim_exp[N:, N:]

    loss = -(torch.log(sim_1_2.sum(dim=1) / (sim_1_1.sum(dim=1) + sim_1_2.sum(dim=1))).mean() +
             torch.log(sim_2_1.sum(dim=1) / (sim_2_1.sum(dim=1) + sim_2_2.sum(dim=1))).mean()) / 2
    return loss

In [6]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [7]:
transform = transforms.Compose([
    transforms.ToTensor()
])
train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)

Files already downloaded and verified


In [8]:
resnet_model = ResNet18SimCLR().to(device)

# Optimizer
optimizer = optim.Adam(resnet_model.parameters(), lr=3e-4)

In [9]:
num_epochs = 15
temperature = 0.5
for epoch in range(num_epochs):
    total_loss = 0.0
    bs=0
    for i, (images, _) in enumerate(train_loader):
        images = images.to(device)

        # Forward pass
        images_aug1 = augment(images).to(device)
        images_aug2 = augment(images).to(device)

        features1 = projection_head((resnet_model(images_aug1),), device=device)
        features2 = projection_head((resnet_model(images_aug2),), device=device)

        # Calculate loss
        loss = NTXentLoss(features1, features2, temperature)

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        print(f"batch [{bs}],Epoch going [{epoch}], Loss: {total_loss}")
        bs+=1
    # Print epoch loss
    print(f"epoch[{epoch+1}/{num_epochs}], Loss: {total_loss / len(train_loader):.4f}")
   
    #model checkpoint
    checkpoint_path = 'resnet_simclr_checkpoint.pth'

    # Save the model checkpoint
    torch.save({
    'epoch': epoch,
    'model_state_dict': resnet_model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': total_loss / len(train_loader),
    }, checkpoint_path)

batch [0],Epoch going [0], Loss: 1.8111889362335205
batch [1],Epoch going [0], Loss: 3.5674052238464355
batch [2],Epoch going [0], Loss: 5.431925535202026
batch [3],Epoch going [0], Loss: 7.099295139312744
batch [4],Epoch going [0], Loss: 8.845873594284058
batch [5],Epoch going [0], Loss: 10.483515977859497
batch [6],Epoch going [0], Loss: 12.029024362564087
batch [7],Epoch going [0], Loss: 13.698266744613647
batch [8],Epoch going [0], Loss: 15.2481369972229
batch [9],Epoch going [0], Loss: 16.820189237594604
batch [10],Epoch going [0], Loss: 18.484810948371887
batch [11],Epoch going [0], Loss: 20.04108726978302
batch [12],Epoch going [0], Loss: 21.51816189289093
batch [13],Epoch going [0], Loss: 23.08226776123047
batch [14],Epoch going [0], Loss: 24.571635961532593
batch [15],Epoch going [0], Loss: 26.0511292219162
batch [16],Epoch going [0], Loss: 27.569857954978943
batch [17],Epoch going [0], Loss: 29.075791001319885
batch [18],Epoch going [0], Loss: 30.685845494270325
batch [19],Ep

In [10]:

# Initialize parameters for the linear layer
num_features = 512  # This is typical for ResNet-18's final layer output
num_classes = 10
linear_layer = nn.Linear(num_features, num_classes).to(device)

# Optimizer for the linear layer
optimizer = torch.optim.Adam(linear_layer.parameters(), lr=0.001)


In [11]:
def apply_linear_classifier(features, linear_layer):
    return linear_layer(features)


In [16]:
criterion = nn.CrossEntropyLoss()
outputs=[]
for epoch in range(num_epochs):
    total_loss = 0
    correct = 0
    total = 0
    
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        
        # Assuming 'resnet_model' is your pre-trained ResNet model and it's already loaded
        with torch.no_grad():
            features = resnet_model(images)
        
        # Reset gradient
        optimizer.zero_grad()
        
        # Apply linear classifier
        outputs = apply_linear_classifier(features, linear_layer)
        
        # Calculate loss
        loss = criterion(outputs, labels)
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        total_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    
    # Print statistics
    print(f"Epoch [{epoch+1}/{num_epochs}] Loss: {total_loss / len(train_loader):.4f} Acc: {100 * correct / total:.2f}%")


Epoch [1/15] Loss: 1.9679 Acc: 27.76%
Epoch [2/15] Loss: 1.9711 Acc: 27.79%
Epoch [3/15] Loss: 1.9692 Acc: 27.96%
Epoch [4/15] Loss: 1.9698 Acc: 27.92%
Epoch [5/15] Loss: 1.9674 Acc: 28.06%
Epoch [6/15] Loss: 1.9662 Acc: 28.13%
Epoch [7/15] Loss: 1.9665 Acc: 28.02%
Epoch [8/15] Loss: 1.9685 Acc: 28.11%
Epoch [9/15] Loss: 1.9638 Acc: 27.90%
Epoch [10/15] Loss: 1.9650 Acc: 28.17%
Epoch [11/15] Loss: 1.9685 Acc: 27.81%
Epoch [12/15] Loss: 1.9706 Acc: 27.99%
Epoch [13/15] Loss: 1.9675 Acc: 27.85%
Epoch [14/15] Loss: 1.9635 Acc: 28.28%
Epoch [15/15] Loss: 1.9661 Acc: 27.83%


In [17]:
# Define the path where you want to save the model
model_save_path_linear = 'linear_classifier_2.pth'

# Save the model state dictionary
torch.save(linear_layer.state_dict(),model_save_path_linear)


In [18]:
# Save model and optimizer state
checkpoint_linear = {
    'model_state_dict': linear_layer.state_dict(),
    'optimizer_state_dict': optimizer.state_dict()
}
torch.save(checkpoint_linear, 'linear_classifier_checkpoint_2.pth')


In [ ]:

def load_checkpoint(model_path, model, optimizer):
    checkpoint = torch.load(model_path)
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
    return model, optimizer

# Assuming the classifier and optimizer have been defined:
num_features = 512  # Assuming this is known
num_classes = 10    # For CIFAR-10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

classifier = nn.Linear(num_features, num_classes).to(device)
optimizer = optim.Adam(classifier.parameters(), lr=0.001)

# Load model and optimizer from checkpoint
classifier, optimizer = load_checkpoint('linear_classifier_checkpoint_2.pth', classifier, optimizer)


In [22]:

# Assuming num_features and num_classes are known
num_features = 512  # Example feature size from a ResNet18 model
num_classes = 10    # Number of classes for CIFAR-10

# Define the classifier structure
classifier = nn.Linear(num_features, num_classes)
checkpoint_check = torch.load('linear_classifier_checkpoint_2.pth')

# Correctly load the model's state dictionary
classifier.load_state_dict(checkpoint_check['model_state_dict'])

# Assuming you are using a GPU if available
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
classifier.to(device)


Linear(in_features=512, out_features=10, bias=True)

In [24]:
print("Classifier Summary:")
print(f"Input features: 512")
print(f"Output features: 10")
print(f"Number of parameters: {sum(p.numel() for p in classifier.parameters() if p.requires_grad)}")



Classifier Summary:
Input features: 512
Output features: 10
Number of parameters: 5130


In [25]:
test_transform = transforms.Compose([
    transforms.ToTensor()
])

In [26]:

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True,transform=test_transform)
test_loader = DataLoader(test_dataset, batch_size=28, shuffle=False, num_workers=4)
test_size = len(test_dataset)
print(f"Size of the test set: {test_size} samples")

Files already downloaded and verified
Size of the test set: 10000 samples


In [29]:
def test_model(model, test_loader, criterion):
    model.eval()  # Set the model to evaluation mode
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    total_loss_test = 0.0
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            total_loss_test += loss.item()
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    accuracy = correct / total
    avg_loss = total_loss / len(test_loader)
    print(f"loss on test set: {total_loss_test:.4f}")
    return accuracy, avg_loss

In [32]:
from sklearn.metrics import accuracy_score
criterion = nn.CrossEntropyLoss()
test_accuracy,test_avg_loss = test_model(resnet_model, test_loader,criterion)
print(f"Accuracy on test set: {test_accuracy:.4f}")

loss on test set: 2455.1657
Accuracy on test set: 0.0022


In [37]:
pretrained_dict = torch.load('resnet_simclr_checkpoint.pth')
pretrained_model = pretrained_dict['model_state_dict']
pretrained_model.fc = nn.Identity()


In [40]:
print(pretrained_dict.keys())

dict_keys(['epoch', 'model_state_dict', 'optimizer_state_dict', 'loss'])


In [45]:
from torchsummary import summary
resnet_model.load_state_dict(pretrained_model)
# Assuming 'pretrained_model' is your pretrained model object
summary(resnet_model, input_size=(3, 32, 32))  

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 64, 16, 16]           9,408
       BatchNorm2d-2           [-1, 64, 16, 16]             128
              ReLU-3           [-1, 64, 16, 16]               0
         MaxPool2d-4             [-1, 64, 8, 8]               0
            Conv2d-5             [-1, 64, 8, 8]          36,864
       BatchNorm2d-6             [-1, 64, 8, 8]             128
              ReLU-7             [-1, 64, 8, 8]               0
            Conv2d-8             [-1, 64, 8, 8]          36,864
       BatchNorm2d-9             [-1, 64, 8, 8]             128
             ReLU-10             [-1, 64, 8, 8]               0
       BasicBlock-11             [-1, 64, 8, 8]               0
           Conv2d-12             [-1, 64, 8, 8]          36,864
      BatchNorm2d-13             [-1, 64, 8, 8]             128
             ReLU-14             [-1, 6

In [46]:
if 'fc' in pretrained_model:
    del pretrained_model['fc']

In [48]:
trained_model = resnet_model  # Replace YourModel() with the class name of your model
trained_model.load_state_dict(pretrained_model)

<All keys matched successfully>

In [49]:
state_dict_keys = trained_model.state_dict().keys()
print(state_dict_keys)

odict_keys(['conv1.weight', 'bn1.weight', 'bn1.bias', 'bn1.running_mean', 'bn1.running_var', 'bn1.num_batches_tracked', 'layer1.0.conv1.weight', 'layer1.0.bn1.weight', 'layer1.0.bn1.bias', 'layer1.0.bn1.running_mean', 'layer1.0.bn1.running_var', 'layer1.0.bn1.num_batches_tracked', 'layer1.0.conv2.weight', 'layer1.0.bn2.weight', 'layer1.0.bn2.bias', 'layer1.0.bn2.running_mean', 'layer1.0.bn2.running_var', 'layer1.0.bn2.num_batches_tracked', 'layer1.1.conv1.weight', 'layer1.1.bn1.weight', 'layer1.1.bn1.bias', 'layer1.1.bn1.running_mean', 'layer1.1.bn1.running_var', 'layer1.1.bn1.num_batches_tracked', 'layer1.1.conv2.weight', 'layer1.1.bn2.weight', 'layer1.1.bn2.bias', 'layer1.1.bn2.running_mean', 'layer1.1.bn2.running_var', 'layer1.1.bn2.num_batches_tracked', 'layer2.0.conv1.weight', 'layer2.0.bn1.weight', 'layer2.0.bn1.bias', 'layer2.0.bn1.running_mean', 'layer2.0.bn1.running_var', 'layer2.0.bn1.num_batches_tracked', 'layer2.0.conv2.weight', 'layer2.0.bn2.weight', 'layer2.0.bn2.bias', '

In [51]:
#print(trained_model)

# Check if there's a classification layer in the forward method


# Check if there's a classification layer in the named modules of the model
print("Modules:")
for name, module in trained_model.named_modules():
    print(name, module)

Modules:
 ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplac